# Setup and Import

In [ ]:
import pandas as pd
import sqlite3
from tabulate import tabulate
import calendar

# --- Paths ---
csv_path = "/content/drive/MyDrive/Colab Notebooks/Project 2 Data - VS.csv"
db_path = "/content/drive/MyDrive/Colab Notebooks/project2.db"

# --- Load CSV ---
df = pd.read_csv(csv_path, dtype=str)

# --- Remove fully empty rows (extra NaN rows) ---
df = df.dropna(how='all')

# Optional: ensure only the first 8 rows are kept, if there might be extra blanks
df = df.iloc[:8]

# --- Save to SQLite ---
# --- Load CSV into SQLite ---
conn = sqlite3.connect(db_path)
cur = conn.cursor()
df.to_sql("sales", conn, if_exists="replace", index=False)

# --- Show table to verify ---
final_df = pd.read_sql("SELECT * FROM sales;", conn)
print(tabulate(final_df.fillna(""), headers="keys", tablefmt="grid", showindex=False))

# Data Cleaning


In [ ]:
# Section 1: Remove duplicate rows across all columns
cur.execute("""
DELETE FROM sales
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM sales
    GROUP BY Order_ID, Customer_Name, Email, Phone, Product_Category, Order_Date, Revenue, "Discount (%)"
);
""")
conn.commit()

# Section 2: Replace missing emails
cur.execute("""
UPDATE sales
SET Email = 'not_provided@email.com'
WHERE Email IS NULL OR TRIM(Email) = '';
""")
conn.commit()

# Section 3: Replace missing phone numbers
cur.execute("""
UPDATE sales
SET Phone = '0000000000'
WHERE Phone IS NULL OR TRIM(Phone) = '';
""")
conn.commit()

# Section 4: Remove duplicates by Order_ID
cur.execute("""
DELETE FROM sales
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM sales
    GROUP BY Order_ID
);
""")
conn.commit()

# Section 5: Remove duplicates by Customer_Name, Email, Phone, Product_Category, Order_Date, Revenue, Discount (%)
cur.execute("""
DELETE FROM sales
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM sales
    GROUP BY Customer_Name, Email, Phone, Product_Category, Order_Date, Revenue, "Discount (%)"
);
""")
conn.commit()

# Section 6: Fix missing discounts
cur.execute("""
UPDATE sales
SET "Discount (%)" = '0.00%'
WHERE "Discount (%)" IS NULL OR TRIM("Discount (%)") = '';
""")
conn.commit()

# Section 7: Ensure Order_Date is stored as proper date format (YYYY-MM-DD)
cur.execute("""
UPDATE sales
SET Order_Date =
    CASE
        WHEN Order_Date LIKE '____-__-__' THEN Order_Date  -- already in correct format
        WHEN Order_Date LIKE '__/__/____' THEN
            substr(Order_Date, 7, 4) || '-' || substr(Order_Date, 1, 2) || '-' || substr(Order_Date, 4, 2)
        WHEN Order_Date LIKE '_/__/____' THEN
            substr(Order_Date, 6, 4) || '-0' || substr(Order_Date, 1, 1) || '-' || substr(Order_Date, 3, 2)
        WHEN Order_Date LIKE '__/_/____' THEN
            substr(Order_Date, 7, 4) || '-' || substr(Order_Date, 1, 2) || '-0' || substr(Order_Date, 4, 1)
        WHEN Order_Date LIKE '_/_/____' THEN
            substr(Order_Date, 5, 4) || '-0' || substr(Order_Date, 1, 1) || '-0' || substr(Order_Date, 3, 1)
        WHEN Order_Date LIKE '__-__-____' THEN
            substr(Order_Date, 7, 4) || '-' || substr(Order_Date, 4, 2) || '-' || substr(Order_Date, 1, 2)
        ELSE Order_Date
    END;
""")
conn.commit()


# Section 8: Ensure Phone is stored as text
cur.execute("""
UPDATE sales
SET Phone = CAST(Phone AS TEXT);
""")
conn.commit()

# Section 9: Check final cleaned table
cur.execute("SELECT * FROM sales;")
rows = cur.fetchall()
columns = [description[0] for description in cur.description]

print(tabulate(rows, headers=columns, tablefmt="grid"))

# Exporting cleaned data as excel file


In [ ]:
# Run your SQL query to fetch cleaned data
cur.execute("SELECT * FROM sales;")
rows = cur.fetchall()
columns = [description[0] for description in cur.description]

# Convert result to a pandas DataFrame
cleaned_df = pd.DataFrame(rows, columns=columns)

# Export DataFrame to Excel
output_path = "/content/cleaned_sales.xlsx"  # file will be saved in Colab's file system
cleaned_df.to_excel(output_path, index=False)

print(f"✅ Cleaned data exported to: {output_path}")

# Aggregation

# Query 1: Total revenue per product category

In [ ]:
query = """
SELECT
    Product_Category,
    SUM(CAST(Revenue AS FLOAT)) AS Total_Revenue
FROM sales
GROUP BY Product_Category
ORDER BY Total_Revenue DESC;
"""

# Run query in SQLite
total_revenue_result_df = pd.read_sql(query, conn)

# Display results in a nice table
from tabulate import tabulate
print(tabulate(total_revenue_result_df, headers="keys", tablefmt="grid", showindex=False))

# Exporting Query 1 Data to Excel

In [ ]:
cur.execute(query)
rows = cur.fetchall()
columns = [description[0] for description in cur.description]

# Convert result to a pandas DataFrame
total_revenue_df = pd.DataFrame(rows, columns=columns)

# Export DataFrame to Excel
output_path = "/content/total_revenue_by_category.xlsx"  # file will be saved in Colab's file system
total_revenue_df.to_excel(output_path, index=False)

print(f"✅ Total revenue by category exported to: {output_path}")

# Query 2: Average discount per customer segment

In [ ]:
query = """
SELECT Product_Category, AVG("Discount (%)") AS Avg_Discount
FROM Sales
GROUP BY Product_Category
"""
# Run query in SQLite
avg_discount_result_df = pd.read_sql(query, conn)

# Display results in a nice table
from tabulate import tabulate
print(tabulate(avg_discount_result_df, headers="keys", tablefmt="grid", showindex=False))

# Exporting Query 2 Data to Excel

In [ ]:
cur.execute(query)
rows = cur.fetchall()
columns = [description[0] for description in cur.description]

# Convert result to a pandas DataFrame
avg_discount_df = pd.DataFrame(rows, columns=columns)

# Export DataFrame to Excel
output_path = "/content/avg_discount_by_category.xlsx"  # file will be saved in Colab's file system
avg_discount_df.to_excel(output_path, index=False)

print(f"✅ Average discount by category exported to: {output_path}")

# Query 3: Monthly Sales Trends


In [ ]:
query = """
SELECT strftime('%m', Order_Date) AS Month_Num,
       SUM(CAST(Revenue AS REAL)) AS Total_Sales
FROM Sales
WHERE Order_Date IS NOT NULL AND TRIM(Order_Date) != ''
GROUP BY strftime('%m', Order_Date)
ORDER BY Month_Num;
"""

# Load into DataFrame
monthly_sales_result_df = pd.read_sql(query, conn)

# Drop rows where Month_Num is None just in case
monthly_sales_result_df = monthly_sales_result_df.dropna(subset=['Month_Num'])

# Convert numeric month to month name
monthly_sales_result_df['Month'] = monthly_sales_result_df['Month_Num'].apply(lambda x: calendar.month_name[int(x)])

# Reorder columns for final display
monthly_sales_result_df = monthly_sales_result_df[['Month', 'Total_Sales']]

# Display nicely
print(tabulate(monthly_sales_result_df, headers='keys', tablefmt='grid', showindex=False))

# Exporting Query 3 Data to Excel

In [ ]:
cur.execute(query)
rows = cur.fetchall()
columns = [description[0] for description in cur.description]

# Convert result to a pandas DataFrame
monthly_sales_df = pd.DataFrame(rows, columns=columns)

# Drop rows where Month_Num is None just in case
monthly_sales_df = monthly_sales_df.dropna(subset=['Month_Num'])

# Convert numeric month to month name
monthly_sales_df['Month'] = monthly_sales_df['Month_Num'].apply(lambda x: calendar.month_name[int(x)])

# Reorder columns for final display
monthly_sales_df = monthly_sales_df[['Month', 'Total_Sales']]

# Export DataFrame to Excel
output_path = "/content/monthly_sales.xlsx"  # file will be saved in Colab's file system
monthly_sales_df.to_excel(output_path, index=False)

print(f"✅ Monthly sales exported to: {output_path}")